# CA273 - Data Processing and Cleaning Assignment - Adam Tegart   

The following is my steps taken to bring the relevent datasets into one dataframe.   
Each data type and the way I integrated it is detailed in full.

Note: The paths used in this notebook are relevant to my file locations, so may need to be changed. All externally processed files used are included in the data/processed/intermediate folder. The 2010 and jan-jun 2020 files were not changed externally.

In [1]:
import pandas as pd

## Formatting the 2020 CSV Dataset

CSV files can be easily read into Pandas as seen below, but one issue with these files was a duplicate month. In the CSV files for July to December the month of June was also included. This, if ignored, would lead to 2 sets of all values in the month of June. I manually went into the csv for Jul-Dec and deleted this dulplicate data.

Doing a sanity check left me scracthing my head as there are only 4367 rows from Jan to Jun and I calculated there should be 4368. This was as a result of day-light savings and the clocks went forward on the 29th of March, checking my dataset this was true. 

In [2]:
first_20 = pd.read_csv("OneDrive/Desktop/A3/a3-2020-jan-jun.csv")
first_20.head()

,Date and Time,OConnell St Outside Pennys,OConnell St Outside Clerys,Mary Street,Capel Street,Aston Quay,Grafton Street at CompuB,Talbot Street North,Doilier Street Burgh Quay,Dawson Street Replacement,...,College Green Bank Of Ireland,Henry Street,Westmoreland Street East,Dawson Street,Liffey Street,Westmoreland Street West,Grafton Street,Bachelors Walk,College Green at Church Lane,College Green - Dame St Side
0,01/01/2020 00:00,1662,2195,60,277,1600,3481,109,994,18,...,893,NaN,1577,NaN,1180,2593,1516.0,868,1494,1797
1,01/01/2020 01:00,1160,990,33,214,858,1870,53,745,7,...,1091,NaN,566,NaN,1133,1917,1169.0,727,676,594
2,01/01/2020 02:00,1110,856,51,219,3499,1626,71,710,15,...,1202,NaN,474,NaN,898,1889,1097.0,930,523,429
3,01/01/2020 03:00,1095,842,44,176,1804,1280,49,1065,8,...,1115,NaN,410,NaN,595,2101,989.0,561,520,365
4,01/01/2020 04:00,666,351,25,98,2562,824,40,768,9,...,1140,NaN,120,NaN,219,873,708.0,269,238,176


## Correcting columns

To allow the datasets to join correctly I will normalize the columns of this dataset as seen below. This involved removing a space in the columns names which I will talk about again in the cleaning phase. Now that the datasets have the same columns they can be appended and pandas will know that the columns are the same.

In [3]:
second_20 = pd.read_csv("OneDrive/Desktop/A3/a3-2020-jul-sep-fixed.csv")
second_20.rename(columns={'OConnell St Outside Clerys ':'OConnell St Outside Clerys',
                          'OConnell St Outside Pennys ':'OConnell St Outside Pennys'}, inplace = True)
second_20.head()

,Date and Time,OConnell St Outside Pennys,OConnell St Outside Clerys,Mary Street,Capel Street,Aston Quay,Grafton Street at CompuB,Talbot Street North,Doilier Street Burgh Quay,Dawson Street Replacement,...,College Green Bank Of Ireland,Henry Street,Westmoreland Street East,Dawson Street,Liffey Street,Westmoreland Street West,Grafton Street,Bachelors Walk,College Green at Church Lane,College Green - Dame St Side
0,01/07/2020 00:00,89,92,NaN,30,151,75,32,201,26,...,98,200,14,NaN,28,99,NaN,11,43,34
1,01/07/2020 01:00,36,26,NaN,28,58,385,11,180,20,...,111,338,27,NaN,5,423,NaN,7,12,12
2,01/07/2020 02:00,18,38,NaN,39,14,782,16,59,2,...,53,294,10,NaN,9,119,NaN,23,14,6
3,01/07/2020 03:00,13,10,NaN,65,37,15,5,15,10,...,25,43,1,NaN,5,26,NaN,3,3,3
4,01/07/2020 04:00,10,17,NaN,50,7,15,2,11,6,...,20,83,3,NaN,4,12,NaN,0,6,4


In [4]:
new_2020 = first_20.append(second_20)

new_2020.columns=['Date and Time', 'O\'Connell St Outside Pennys',
       'O\'Connell St Outside Clerys', 'Mary Street', 'Capel Street',
       'Aston Quay', 'Grafton Street @ CompuB', 'Talbot Street North',
       'Doilier Street, Burgh Quay', 'Dawson Street Replacement',
       'Dame Street', 'Talbot Street South', 'O\'Connell St, Parnell St @ AIB',
       'Grafton Street / Nassau Street / Suffolk Street',
       'College Green, Bank Of Ireland', 'Henry Street',
       'Westmoreland Street East', 'Dawson Street', 'Liffey Street',
       'Westmoreland Street West', 'Grafton Street', 'Bachelors Walk',
       'College Green @ Church Lane', 'College Green - Dame St Side']

## Formatting the 2019 XLSX Dataset

This dataset contains the timestamps every 15 mins as opposed to every hour, this is an issue I need to address.
To correct this I will convert the xlsx files into csv and use a python program to format the data the way I need it.

### My Python program to sum every row in between one hour and the next whole hour to normalize the data

#### I'm sure there is an easier way, but I decided that the most time efficient approach for me was to use something I understood and could manipulate.
_______________________________________________________________________________________________________________________
        
        import sys

        file = sys.argv[1]  #take the file name as a command line argument

        with open(file, "r") as f:
          top = f.readline()  #read the first line with column headings.
          print(top.strip())
          curr_set = 0
          n = top.strip().split(",")
          tot = [0] * (len(n) - 4)  #This will give a list the length of all columns, the split gave too many
          time = None               #as it split names of places that had a comma in them. eg "College Green, Bank Of Ire.."
          for line in f:
            line = line.strip().split(",")
            time_parts = line[0].strip().split(":")  #Every line in the file is split on comma

            if time_parts[1] == "00":   #This was implemented to make sure that even if only 3
              curr_set = 0              #3 of the 4 quaterly measurements were present it would
            elif time_parts[1] == "15": #disrupt the sequence.
              curr_set = 1
            elif time_parts[1] == "30":
              curr_set = 2
            elif time_parts[1] == "45":
              curr_set = 3

            for i in range(0, len(line) - 1):  #cycle through every element in the line
              if time == None:
                time = time_parts[0].replace("-", "/") + ":00"  #if no time value is saved for the current set, save one

              if line[i + 1].strip() != "":  #if there is an entry present proceed
                tot[i] += int(line[i + 1].strip())  #add the entry to its spot in tot


            curr_set += 1  #we are one further through a set of 4

            if curr_set == 4:  #if we have read a full set we need to output a line.
              
              if date_orientation == 1:
                parts = time.strip().split("/")  #If the dates are in the wrong format, switch the day and month
                time = ("/").join([parts[1], parts[0], parts[2]])

              for i in range(0, len(tot)):

                tot[i] = str(tot[i])
              print(time + "," + (",").join(tot))  #output a line with the time and all values, comma seperated

              tot = [0] * (len(n) - 4)
              time = None  #rest all attributes for the next iteration
              curr_set = 0


In [5]:
first_19 = pd.read_csv("OneDrive/Desktop/A3/2019-jan-jun.csv")
first_19.head()

,Time,O'Connell St Outside Pennys,O'Connell St Outside Clerys,Mary Street,Capel Street,Aston Quay,Grafton Street @ CompuB,Talbot Street North,"Doilier Street, Burgh Quay",Dawson Street Replacement,...,"College Green, Bank Of Ireland",Henry Street,Westmoreland Street East,Dawson Street,Liffey Street,Westmoreland Street West,Grafton Street,Bachelors Walk,College Green @ Church Lane,College Green - Dame St Side
0,01/01/2019 00:00,1504,1914,163,238,0,0,0,0,0,...,0,597,1670,255,881,1988,140,0,0,0
1,01/01/2019 01:00,1187,885,102,173,0,0,0,0,0,...,0,359,767,209,723,1270,215,0,0,0
2,01/01/2019 02:00,1233,984,63,121,0,0,0,0,0,...,0,317,642,193,1010,1589,210,0,0,0
3,01/01/2019 03:00,1316,935,59,174,0,0,0,0,0,...,0,313,582,178,772,1534,204,0,0,0
4,01/01/2019 04:00,802,390,46,82,0,0,0,0,0,...,0,172,143,35,197,610,88,0,0,0


In [6]:
second_19 = pd.read_csv("OneDrive/Desktop/A3/2019-jul-dec.csv")
second_19.head()

,Time,O'Connell St Outside Pennys,O'Connell St Outside Clerys,Mary Street,Capel Street,Aston Quay,Grafton Street @ CompuB,Talbot Street North,"Doilier Street, Burgh Quay",Dawson Street Replacement,...,"College Green, Bank Of Ireland",Henry Street,Westmoreland Street East,Dawson Street,Liffey Street,Westmoreland Street West,Grafton Street,Bachelors Walk,College Green @ Church Lane,College Green - Dame St Side
0,01/07/2019 00:00,265,165,13,51,142,349,143,68,0,...,232,121,83,22,101,319,330,50,112,69
1,01/07/2019 01:00,144,75,3,23,82,267,18,25,0,...,136,54,55,11,59,169,226,23,50,31
2,01/07/2019 02:00,99,49,1,11,74,150,24,6,0,...,109,38,46,6,34,135,171,49,39,12
3,01/07/2019 03:00,63,42,4,10,54,186,4,7,0,...,122,35,70,6,26,101,165,21,29,9
4,01/07/2019 04:00,92,47,3,15,34,240,11,7,0,...,129,49,42,3,16,114,171,8,16,18


In [7]:
new_2019 = first_19.append(second_19)
new_2019.rename(columns={"Time":"Date and Time"}, inplace=True)
new_2019.head()

,Date and Time,O'Connell St Outside Pennys,O'Connell St Outside Clerys,Mary Street,Capel Street,Aston Quay,Grafton Street @ CompuB,Talbot Street North,"Doilier Street, Burgh Quay",Dawson Street Replacement,...,"College Green, Bank Of Ireland",Henry Street,Westmoreland Street East,Dawson Street,Liffey Street,Westmoreland Street West,Grafton Street,Bachelors Walk,College Green @ Church Lane,College Green - Dame St Side
0,01/01/2019 00:00,1504,1914,163,238,0,0,0,0,0,...,0,597,1670,255,881,1988,140,0,0,0
1,01/01/2019 01:00,1187,885,102,173,0,0,0,0,0,...,0,359,767,209,723,1270,215,0,0,0
2,01/01/2019 02:00,1233,984,63,121,0,0,0,0,0,...,0,317,642,193,1010,1589,210,0,0,0
3,01/01/2019 03:00,1316,935,59,174,0,0,0,0,0,...,0,313,582,178,772,1534,204,0,0,0
4,01/01/2019 04:00,802,390,46,82,0,0,0,0,0,...,0,172,143,35,197,610,88,0,0,0


## Formatting the 2010 JSON Dataset

What can be seen in this dataset below is that the locations at which data is gathered is not the same as that of the 2019 and 2020 datasets.   

Upon extensive googling and viewing the area on Google Maps, I have found that the 3 Grafton St locations below move all the way along the street, from M&S, to the Card Gallery and then finally to where Korkys once was (now Ted Baker).

Comparing this with the Grafton St locations in the newer dataset, which covers from CompuB which is off the end of Grafton St, and then only Grafton St itself is mentioned.

As the 2010 dataset has 3 spots along Grafton Street I believe the best thing I can do is find the average of the 3 locations in 2010 and use this value to match with the Grafton Street column in the 2019-20 datasets. This is because there is no data in the old set refering to the area around CompuB, Grafton street can be accessed from many side streets. As a result, it is not safe to assume that majority of the traffic came down by CompuB as I cannot tell that from the dataset.

The Henry Street at Butlers will become Henry Street in the new dataset.

Lastly, the Clerys on O'Connells street will stay the same. It can also be seen that Easons is beside Pennys on O'Connells St, so these can be interchanged.

In [8]:
only_2010 = pd.read_json("OneDrive/Desktop/A3/a3-2010.json")
only_2010["Grafton Street"] = round(only_2010[["Grafton St at Card Gallery", "Grafton St at M&S", "Grafton St at Korkys"]].mean(axis=1))

new_2010 = only_2010.drop(["Grafton St at Card Gallery", "Grafton St at M&S", "Grafton St at Korkys"], axis=1)

new_2010.rename(columns={"Unnamed: 0":"Date and Time",
                          "O'Connell Street at Clerys":"O'Connell St Outside Clerys",
                          "O'Connell Street at Easons":"O'Connell St Outside Pennys",
                          "Henry Street at Butlers":"Henry Street"}, inplace=True)
new_2010.head()

,Date and Time,O'Connell St Outside Clerys,O'Connell St Outside Pennys,Henry Street,Grafton Street
0,01/01/2010 00:00,852,131,78,595.0
1,01/01/2010 01:00,736,92,44,730.0
2,01/01/2010 02:00,1064,74,28,1259.0
3,01/01/2010 03:00,1646,106,49,1627.0
4,01/01/2010 04:00,598,35,47,571.0


## Formatting the 2008 ODS Dataset

It can be seen that this dataset is not structered in the same was as the other datasets for many reasons.      
       
Each week worth of collected data is on a seperate sheet. In addition to this, no dates are given, just the week from 1-53 and the day of the week. Lastly, the data itself resides within a table for each collection location. This all poses a challenge for neatly processing the data into a single dataframe.     
     
I plan to use python to process this data as I know how to manipulate strings and create a form I can use.    
Firstly however, I must extract the data from ods format into something I can use. I could manually go through each page and save it as a csv, but this would be tedious and wasteful of my time.    

My solution to this was to use Micrsoft Visual Basic Application, which is a layer underneath Excel. From the developers tab in Excel you can open a VSB tab and enter code to tell Excel exactly what to do without the UI. I used the following code to tuen my ods file into 53 csv files, one for each sheet in the ods.

    ** Sub ExportSheetsToCSV()     
       Dim xWs As Worksheet       
       Dim xcsvFile As String    
       For Each xWs In Application.ActiveWorkbook.Worksheets   
           xWs.Copy   
           xcsvFile = CurDir & "\" & xWs.Name & ".csv"   
           Application.ActiveWorkbook.SaveAs Filename:=xcsvFile, _   
           FileFormat:=xlCSV, CreateBackup:=False   
           Application.ActiveWorkbook.Saved = True   
           Application.ActiveWorkbook.Close    
       Next   
    End Sub **

Now with my 53 csv files I created a python script to read these in one at a time, pick and choose the needed data, find the corresponding date and output all the data in one csv. You'll see that I decided to get the average of the 3 Grafton St locations as this will work well with the other datasets later.
    
### My Python program to convert the 53 csv files into one csv file

#### Again, I'm sure there may be an easier solution to this problem. However, I decided to stick to what I know well.
_______________________________________________________________________________________________________________________

    import sys
    from string import punctuation
    import datetime

    mapping = {"O'Connell Street at Clerys":1, #This allows me to know which place in the list a locations data should go
               "Grafton St at Card Gallery":3,
               "Grafton St at M&S":3,
               "O'Connell Street at Easons":2,
               "Henry Street at Butlers":4,
               "Grafton St at Korkys":3}

    print("Date and Time,O'Connell St Outside Clerys,O'Connell St Outside Pennys,Grafton Street,Henry Street")  #columns
    for i  in range(1, 54): #loop for all files
      days = 7 * (i - 1)  # every file covers a week, so at the start of the week this many days have passed.
      week = []
      for j in range(0, (7*24)):
        week.append([None, 0, 0, 0, 0])  #create a list of all days in the week, all 24 hour slots have a list. (168 lists)

      needed_values = [1, 3, 5, 7, 9, 11, 13] #indexes of the data in the in sections
      with open("week_" + str(i) + ".csv", "r") as f:
        for line in f:  #for every line in the file
          add_time = 0

          if "Entrance Name:" in line:  #If we find a location the following data corresponds to it
            line = line.strip().strip(punctuation).split()
            curr = mapping[(" ").join(line[2:])]

          elif line[2:5] == ":00": #if the line has a time we need to read it
            entry = int(line[:2])  #the time entry of this tuple, for the whole week (ie. mon-sun)
            line = line.strip().split(",")

            for k in range(0, len(needed_values)): #while we haven't taken as many values as needed
              if curr != 3:  #If it is not the Grafton St ones we can just assign it.
                week[entry + (k * 24)][curr] = str(line[needed_values[k]])  #add the value to week inits correct spot.
              else:                                      #multiples of 24 change the day, ie 24 sets of data in a day.
                week[entry + (k * 24)][curr] += int(line[needed_values[k]]) #data from Grafton added so we can average.
              if week[entry + (k * 24)][0] == None:
                week[entry + (k * 24)][0] = line[0]  #If we don't have a time for this tuple yet, take the time from line.

        for i in range(0, len(week)):  #Adding the date to each time interval in the week
          new_days = days
          if i // 24 > 0:   #if we go over 24 hrs in a day we are onto the next day, or hours(i) // 24 days into the week.
            new_days = days + i // 24
          d = datetime.datetime(2008, 1, 1) + datetime.timedelta(new_days - 1)  #function finds the date x days after 1/1/08

          date = d.strftime('%d/%m/%Y')
          week[i][0] = date + " " + str(week[i][0])  #Then convert into the needed format

        for i in range(0, len(week)):
          week[i][3] = str(round(week[i][3]/3))

          print(",".join(week[i]))
  
Running this program will produce a set with 371 days as the Monday in week 1 is from last calendar year, and the last 4 days in week 53 are in 2009. This leaves us with 366 days which is typical of a leap year, so we are correct. Now the dataset can be easily converted into a pandas dataframe as seen below.

In [9]:
only_2008 = pd.read_csv("OneDrive/Desktop/A3/2008.csv")
only_2008.head()

,Date and Time,O'Connell St Outside Clerys,O'Connell St Outside Pennys,Grafton Street,Henry Street
0,01/01/2008 00:00,1429,1013,1560,467
1,01/01/2008 01:00,1044,1075,1546,349
2,01/01/2008 02:00,1272,1287,1841,261
3,01/01/2008 03:00,1364,1544,1744,286
4,01/01/2008 04:00,460,405,691,152


In [10]:
df = only_2008.append(new_2010)
df = df.append(new_2019)
df = df.append(new_2020)

In [11]:
df.to_csv(r'OneDrive/Desktop/A3/a3-processed.csv', index = False, na_rep='0')

## Reading in my cleaned file to finish off my assignment

With the file processed and cleaned, there is a couple questions left to answer.
I must get the .describe() of my finalized dataset and also the .info().

In addition to this, I must use the dataset to "Calculate the total number of pedestrians counted at the location nearest to "O'Connell Street at Clerys" during March 2010, March 2019 and March 2020."

First as I have the 2008 set integrated aswell I will include that data too, then I'll output just 2020, 2019 and 2010 after.

In [12]:
final_df = pd.read_csv("OneDrive/Desktop/A3/a3-finalized.csv")

In [13]:
final_df.describe()

,O'Connell St Outside Clerys,O'Connell St Outside Pennys,Grafton Street,Henry Street,Mary Street,Capel Street,Aston Quay,Grafton Street @ CompuB,Talbot Street North,"Doilier Street, Burgh Quay",...,"O'Connell St, Parnell St @ AIB",Grafton Street / Nassau Street / Suffolk Street,"College Green, Bank Of Ireland",Westmoreland Street East,Dawson Street,Liffey Street,Westmoreland Street West,Bachelors Walk,College Green @ Church Lane,College Green - Dame St Side
count,32851.000000,32851.000000,32851.000000,32851.000000,32851.000000,32851.000000,32851.000000,32851.000000,32851.000000,32851.000000,...,32851.000000,32851.000000,32851.000000,32851.000000,32851.000000,32851.000000,32851.000000,3.285100e+04,32851.000000,32851.000000
mean,1335.689325,1574.987185,2026.891114,1855.929956,75.962223,97.786582,423.602691,649.685215,280.042251,281.735533,...,233.759398,138.882591,259.556482,182.252291,77.948616,294.343703,391.354114,2.024130e+02,255.813826,232.516331
std,1348.805644,1955.864093,3110.957072,2199.519385,177.317251,191.034472,775.798612,1189.171127,878.331693,523.371515,...,427.683058,278.791302,470.366251,325.656292,220.427987,565.887166,605.351821,5.541406e+03,480.582710,461.355649
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,253.000000,175.000000,219.000000,140.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
50%,976.000000,766.000000,1004.000000,760.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
75%,1946.000000,2222.000000,2870.000000,3314.000000,31.000000,130.000000,618.000000,724.000000,120.000000,425.000000,...,250.000000,127.000000,346.000000,207.000000,0.000000,300.000000,705.000000,7.600000e+01,258.000000,204.000000
max,16109.000000,11791.000000,338088.000000,13625.000000,1263.000000,5883.000000,4953.000000,12188.000000,10729.000000,5995.000000,...,2141.000000,2455.000000,4632.000000,2852.000000,2047.000000,4700.000000,3280.000000,1.000187e+06,2392.000000,4147.000000


This max for Grafton St seems absurd. Although after inspection, this is the day that christmas lights were turned on in the area for the first time (13/12/2010 at 5pm). So it may be an accurate value, especially considering that people may walk up and down the street and that the data was collected by sensors, not people.

In [14]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32851 entries, 0 to 32850
Data columns (total 24 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Date and Time                                    32851 non-null  object
 1   O'Connell St Outside Clerys                      32851 non-null  int64 
 2   O'Connell St Outside Pennys                      32851 non-null  int64 
 3   Grafton Street                                   32851 non-null  int64 
 4   Henry Street                                     32851 non-null  int64 
 5   Mary Street                                      32851 non-null  int64 
 6   Capel Street                                     32851 non-null  int64 
 7   Aston Quay                                       32851 non-null  int64 
 8   Grafton Street @ CompuB                          32851 non-null  int64 
 9   Talbot Street North                    

In [15]:
final_df["Date"] = pd.to_datetime(final_df["Date and Time"], dayfirst=True) #change the column values into datetime values
all_march_df = final_df[final_df["Date"].dt.month == 3]

In [16]:
print("Over the course of all 4 years recorded a total of {:d} people were recorded passing by the Clerys on O'Connell St".format(
    all_march_df["O'Connell St Outside Clerys"].sum()))

Over the course of all 4 years recorded a total of 3859766 people were recorded passing by the Clerys on O'Connell St


In [17]:
no_2008_df = all_march_df[all_march_df["Date"].dt.year != 2008]
print("Over the course of all years in the set excluding 2008 a total of {:d} people were recorded passing by the Clerys on O'Connell St".format(
    no_2008_df["O'Connell St Outside Clerys"].sum()))

Over the course of all years in the set excluding 2008 a total of 2402536 people were recorded passing by the Clerys on O'Connell St


## Results

It can be seen that over the course of March in 2020 only 416,000 people were recoreded to have passed by Clerys on O'Connell St. By exclusing 2008 from the sum we lost 1.4 million passersby. No Paddy's day in 2020 seems to have taken a heavy toll on the pedestrian figures in 2020. Although with multiple lockdowns and restrictions in place, it is no wonder that only 66,652,586 passersby were recorded in 2020 between jan-sep compared to 117,695,332 in 2019 in the same time period. This is not factoring in the few months here and there that were normal with regards to travel and shopping etc.

Overall, I have found this project a little tedious. Although, looking at the results I feel very satisfied with the final dataset and couldn't help but have a look through these figures and find something interesting. That's the whole purpose of the cleaning! To find the data that can give insight. 